## Задачи для совместного разбора

In [ ]:
!pip install torchtyping

  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

Scalar = TensorType[()]
patch_typeguard()

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [ ]:
class Neuron:
    def __init__(self, n_features: int, bias: float) -> None:
        # <создать атрибуты объекта weights и bias>
        self.weights: TensorType["n_features"] = th.rand(n_features)
        self.bias: float = bias

    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> Scalar:
        return inputs @ self.weights + self.bias


In [ ]:
import torch as th
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])

In [ ]:
neuron = Neuron(n_features=4, bias=0.0)
neuron.forward(inputs)

tensor(4.5098)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ReLU:
    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> TensorType["n_features"]:
        # <реализовать логику ReLU>
        return th.clip(inputs, min=0)

In [ ]:
inputs = th.tensor([1.0, -2.0, 3.0, -4.0])

act = ReLU()
act.forward(inputs)

tensor([1., 0., 3., 0.])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [ ]:
class MSELoss:
    @typechecked
    def forward(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"]
    ) -> Scalar:
        return ((y_pred - y_true)**2).mean()

In [ ]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

In [ ]:
criterion = MSELoss()
loss = criterion.forward(y_pred, y_true)

In [ ]:
loss

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1"></p>

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [ ] Проверено на семинаре

In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

class Linear:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.bias = bias
        self.weights = th.normal(mean=0, std=1, size = (n_features, n_neurons))

    @typechecked
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        return inputs @ self.weights + self.bias*th.tensor([0.6, -0.8, 1.4]) #рандомно придумал biases

In [ ]:
n_neurons = 3
n_features = 4
batch_size = 10
inputs = th.rand(batch_size*n_features).view(batch_size, n_features)
full_con_nn = Linear(n_neurons=n_neurons, n_features=n_features, bias=True)
res = full_con_nn.forward(inputs)
res, res.shape

(tensor([[-0.8958, -0.8287,  2.3835],
         [-0.5842,  0.1199,  2.4194],
         [-0.6234, -1.4394,  1.1850],
         [ 0.6465,  0.4881,  1.8058],
         [-0.4465,  0.8498,  3.0482],
         [ 0.7190, -0.8728,  0.9039],
         [-0.7341,  0.5472,  3.0789],
         [ 0.2042, -0.2248,  1.6952],
         [-0.0566, -0.2374,  1.7337],
         [ 0.4217, -0.7949,  1.0038]]),
 torch.Size([10, 3]))

<p class="task" id="2"></p>

2\. Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

- [ ] Проверено на семинаре

In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

class Linear_2l:
    def __init__(self, n_neurons_l1: int, n_neurons_l2: int, n_features: int,
                 bias_1l: bool = False, bias_2l: bool = False) -> None:

        self.bias_1l = th.rand(n_neurons_l1) if bias_1l else th.zeros(n_neurons_l1)
        self.bias_2l = th.rand(n_neurons_l2) if bias_2l else th.zeros(n_neurons_l2)
        self.weights_1l = th.normal(mean=0, std=1, size = (n_features, n_neurons_l1))
        self.weights_2l = th.normal(mean=0, std=1, size = (n_neurons_l1, n_neurons_l2))

    @typechecked
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        res_1l = inputs @ self.weights_1l + self.bias_1l
        res_2l = res_1l @ self.weights_2l + self.bias_2l
        return res_2l

In [ ]:
n_neurons_l1 = 2
n_neurons_l2 = 7 #тут обязательно 7 по условию
n_features = 6
batch_size = 4
inputs = th.rand(batch_size*n_features).view(batch_size, n_features)
full_con_nn_2l = Linear_2l(n_neurons_l1=n_neurons_l1, n_neurons_l2=n_neurons_l2, n_features=n_features, bias_1l=True, bias_2l=True)
res = full_con_nn_2l.forward(inputs)
res#, res.shape

tensor([[ 4.3778,  3.8988, -1.5195,  0.9480,  1.0634,  1.9334,  2.9248],
        [ 1.4512,  1.6516, -1.1110,  0.1027,  0.2805,  0.6870,  1.1737],
        [ 8.6687,  5.7959,  0.3427,  2.7702,  2.6324,  3.3726,  5.7171],
        [ 7.3554,  6.3539, -2.2326,  1.7375,  1.8090,  3.2485,  4.6793]])

### Создание функций активации

<p class="task" id="3"></p>

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

- [ ] Проверено на семинаре

In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th
t = th.randn(4,3)
t

tensor([[ 0.0729,  0.1240, -0.2653],
        [-0.6858,  0.0802,  0.0487],
        [ 0.0216, -0.0430, -0.4509],
        [-0.0603,  2.2653, -1.6226]])

In [ ]:
class Softmax:
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        # <реализовать логику Softmax>
        return th.exp(inputs) / th.sum(th.exp(inputs), dim=1, keepdim=True)

In [ ]:
act_f = Softmax()
act_f.forward(t)

tensor([[0.3616, 0.3805, 0.2578],
        [0.1910, 0.4109, 0.3981],
        [0.3905, 0.3660, 0.2435],
        [0.0874, 0.8943, 0.0183]])

<p class="task" id="4"></p>

4 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

- [ ] Проверено на семинаре

In [ ]:
class ELU:
    def __init__(self, alpha: float) -> None:
        self.alpha = alpha

    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        return th.where(inputs >= 0, inputs, self.alpha * (th.exp(inputs) - 1))

In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th
t = th.randn(4,3)
t

tensor([[ 0.7398, -0.9962, -0.0978],
        [-0.9853,  0.2535,  1.2366],
        [-1.0912,  1.6241, -1.1559],
        [-0.0855, -0.9903,  0.6626]])

In [ ]:
act_f = ELU(alpha=0.3)
act_f.forward(t)

tensor([[ 0.7398, -0.1892, -0.0279],
        [-0.1880,  0.2535,  1.2366],
        [-0.1993,  1.6241, -0.2056],
        [-0.0246, -0.1886,  0.6626]])

### Создание функции потерь

<p class="task" id="5"></p>

5 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 2 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

- [ ] Проверено на семинаре

In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

class CrossEntropyLoss:
    def forward(
      self,
      y_pred: TensorType["batch", "n_neurons", float],
      y_true: TensorType["batch", float]
    ):
    # <реализовать логику функции потерь>
        loss =  -(y_true*th.log(y_pred)).mean()
        return loss

In [ ]:
class Linear_1l:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.bias = th.rand(n_neurons_l1) if bias else th.zeros(n_neurons_l1)
        self.weights = th.normal(mean=0, std=1, size = (n_features, n_neurons))

    @typechecked
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        res = inputs @ self.weights + self.bias
        return res

In [ ]:
n_neurons = 2
n_features = 4
batch_size = 5
inputs = th.rand(batch_size*n_features).view(batch_size, n_features)
full_con_nn = Linear_1l(n_neurons=n_neurons, n_features=n_features, bias=True)
res_nn = full_con_nn.forward(inputs)
res_nn

tensor([[-0.0846,  1.7593],
        [ 0.1109,  1.8966],
        [ 0.6373,  1.5660],
        [ 0.5746,  0.4391],
        [-0.2582,  1.2159]])

In [ ]:
act_f = Softmax()
res_act = act_f.forward(res_nn)
res_act

tensor([[0.1366, 0.8634],
        [0.1436, 0.8564],
        [0.2832, 0.7168],
        [0.5338, 0.4662],
        [0.1863, 0.8137]])

In [ ]:
y_true = th.randn(batch_size).view(5,1)
y_true

tensor([[ 0.6510],
        [ 0.7342],
        [-2.5981],
        [ 0.0971],
        [ 0.0945]])

In [ ]:
criterion = CrossEntropyLoss()
loss = criterion.forward(res_act, y_true)
loss

tensor(-0.0899)

<p class="task" id="6"></p>

6 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [ ] Проверено на семинаре

In [ ]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

class Scalar:
    pass

class MSERegularized:
    def __init__(self, lambda_: float) -> None:
        self.lambda_ = lambda_

    def data_loss(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"],
    ) -> Scalar:
        # <подсчет первого слагаемого из формулы>
        return ((y_true-y_pred)**2).mean()

    def reg_loss(self, weights: TensorType["batch", 1])  -> Scalar:
        # <подсчет второго слагаемого из формулы>
        return self.lambda_ * sum(weights**2)

    def forward(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"],
        weights: TensorType["batch", 1],
    ) -> Scalar:
        return self.data_loss(y_pred, y_true) + self.lambda_ * self.reg_loss(weights)

In [ ]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])
weights = th.tensor([0.1, 0.7, -0.2])
criterion = MSERegularized(0.2)
loss = criterion.forward(y_pred, y_true, weights)
loss

tensor(0.6883)